<a href="https://colab.research.google.com/github/ab-dul-aziz/NGC/blob/main/Latihan_LC2_P0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kisi-kisi LC2**

Dataset yang digunakan pada latihan ini berasal dari bigquery public data. Lakukan koneksi Google Colaboratory dengan BigQuery menggunakan syntax berikut:

```
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "hckv8-95102" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
```

lalu lakukan query pada notebook menggunakan cara berikut:

```
df_1 = client.query('''
SELECT *
FROM table_name`
''').to_dataframe()

df_1
```

---

Dataset yang digunakan merupakan `record perjalanan peminjam sepeda`, dapat diakses pada:

- **Project_id_Database**: ftds-hacktiv8-project
- **Dataset**            : practice
- **Table**              : bike_rent

Lakukan pengolahan SQL langsung melalui Google Colaboratory yang telah dikoneksikan pada BigQuery

---
Pada notebook ini terdapat 2 section, **SQL** dan **Pandas**.

Kerjakan masing-masing section sesuai dengan ketentuan di bawah:

## A. **SQL**

  >**Jawab masing-masing soal dalam 1 query.**

  1. Berapa banyak label unique dari `bike_type`?  
  2. Apa saja nama label unique dari tipe sepeda?
  3. Berapa banyak total peminjaman sepeda pada tahun 2016 dan 2017?
  4. Dimana station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016?
  5. Berapa nilai minimum, maximum, dan rata-rata dari durasi perjalanan pada keseluruhan data?
  6. Berapa rata-rata durasi perjalanan dari subscriber_type `Explorer` dan `Weekender`?
  7. Tampilkan entry data yang memiliki `duration_time` diatas 4.25 jam. (*Gunakan subquery untuk menjawab pertanyaan ini dengan meng-convert duration_time dari menit ke hour*).

## B. **Pandas**
  >**Data yang akan diolah menggunakan Pandas wajib diambil menggunakan Query berikut:**
  ```
  SELECT *
  FROM `ftds-hacktiv8-project.practice.bike_rent`
  ```
  
  >Simpan data yang telah diambil ke dalam variabel `df` (penamaan variabel dibebaskan), **lakukan pengolah di bawah ini menggunakan library Pandas**:

  1. Cek kesesuaian tipe data dan keberadaan missing values dengan `.info()`

  2. Buat kolom baru bernama `quarter`.
  
  Lakukan ekstraksi waktu (quarter dalam tahun) berdasarkan kolom `start_time` dengan ketentuan berikut:

<div align="center">

  |Keterangan Waktu|Bulan|
  |--|--|
  |Q1|Januari, Februari, Maret|
  |Q2|April, Mei, Juni|
  |Q3|Juli, Agustus, September|
  |Q4|Oktober, November, Desember|

</div>

  3. Tampilkan data perjalanan dari tipe member `Local365` dengan durasi perjalanan diatas 1 jam *`(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)`*.

  4. Berapa rata-rata durasi perjalanan dari tipe pelanggan `Walk Up` dengan tipe sepeda `classic` pada bulan `Maret` berdasarkan data yang diambil? *`(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)`*.

>**Notes:**
>- Dalam menjawab soal Pandas no. 2, **tidak boleh** menggunakan ekstraksi waktu `Series.dt.quarter`
- Lakukan `looping` dan `conditional` dengan memanfaatkan ekstraksi bulan `Series.dt.month` terhadap kolom `start_time`, pastikan tipe datanya ialah `datetime` agar dapat dilakukan ekstraksi.
- Simpan hasil looping ke dalam suatu variabel, dan buat kolom baru berisikan variabel sebelumnya.



###**SQL**

In [5]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-438003" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


#####1. Berapa banyak label unique dari bike_type?

In [11]:
# working area
df1 = client.query('''
SELECT COUNT(DISTINCT bike_type) AS count_unique_bike_type
FROM `ftds-hacktiv8-project.practice.bike_rent`;
''').to_dataframe()

df1

,count_unique_bike_type
0,2


#####2. Apa saja nama label unique dari tipe sepeda?

In [12]:
# working area
df2 = client.query('''
SELECT DISTINCT (bike_type) AS unique_bike_type
FROM `ftds-hacktiv8-project.practice.bike_rent`;
''').to_dataframe()

df2

,unique_bike_type
0,classic
1,electric


#####3. Berapa banyak total peminjaman sepeda pada tahun 2016 dan 2017?

In [13]:
# working area
df3 = client.query('''
SELECT
    COUNT(*) AS total_trip,
    EXTRACT(YEAR FROM start_time) AS year
FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE EXTRACT(YEAR FROM start_time) IN(2016, 2017)
GROUP BY 2;
''').to_dataframe()

df3

,total_trip,year
0,4539,2017
1,3454,2016


#####4. Dimana station yang paling banyak dikunjungi sebagai titik keberangkatan pada tahun 2016?

In [14]:
# working area
df4 = client.query('''
SELECT
    start_station_name,
    COUNT(*) AS most_visited
FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE EXTRACT(YEAR FROM start_time) = 2016
GROUP BY start_station_name
ORDER BY most_visited DESC
LIMIT 1;
''').to_dataframe()

df4

,start_station_name,most_visited
0,Bullock Museum @ Congress & MLK,394


#####5. Berapa nilai minimum, maximum, dan rata-rata dari durasi perjalanan pada keseluruhan data?

In [15]:
# working area

df5 = client.query('''
SELECT
    MIN(duration_minutes) AS min_duration,
    MAX(duration_minutes) AS max_duration,
    AVG(duration_minutes) AS avg_duration
FROM `ftds-hacktiv8-project.practice.bike_rent`;
''').to_dataframe()

df5

,min_duration,max_duration,avg_duration
0,2,2218,35.0995


#####6. Berapa rata-rata durasi perjalanan dari subscriber_type `Student Membership` dan `Local31`?

In [16]:
# working area
df6 = client.query('''
SELECT
    subscriber_type,
    AVG(duration_minutes) AS avg_duration
FROM `ftds-hacktiv8-project.practice.bike_rent`
WHERE subscriber_type = 'Local30'
    OR subscriber_type LIKE '%Member%'
GROUP BY 1;
''').to_dataframe()

df6

,subscriber_type,avg_duration
0,Local30,16.021672
1,U.T. Student Membership,39.000000


#####7. Tampilkan entry data yang memiliki `duration_time` diatas 4.25 jam. (*Gunakan subquery untuk menjawab pertanyaan ini dengan meng-convert duration_time dari menit ke hour*).

In [17]:
# working area
df7 = client.query('''
SELECT *
FROM  (SELECT *,
              ROUND(duration_minutes/60, 2) AS duration_hour
      FROM `ftds-hacktiv8-project.practice.bike_rent`
      WHERE (duration_minutes/60) > 4.25);
''').to_dataframe()

df7

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,duration_hour
0,13255686,Walk Up,539,classic,2016-12-25 11:17:16+00:00,Pease Park,Pease Park,351,5.85
1,13255691,Walk Up,453,classic,2016-12-25 11:18:17+00:00,Pease Park,Pease Park,350,5.83
2,14534164,Walk Up,664,classic,2017-04-21 20:46:17+00:00,Davis at Rainey Street,Pease Park,783,13.05
3,14534142,Walk Up,275,classic,2017-04-21 20:41:54+00:00,Davis at Rainey Street,Pease Park,789,13.15
4,11965934,Walk Up,453,classic,2016-09-17 11:55:23+00:00,Bullock Museum @ Congress & MLK,Lavaca & 6th,325,5.42
...,...,...,...,...,...,...,...,...,...
86,14699898,Walk Up,262,classic,2017-05-06 09:08:08+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,350,5.83
87,9516320,Walk Up,157,classic,2016-04-11 13:19:09+00:00,Lavaca & 6th,State Capitol @ 14th & Colorado,643,10.72
88,14465838,Walk Up,871,classic,2017-04-16 12:39:51+00:00,South Congress & Academy,State Capitol @ 14th & Colorado,684,11.40
89,15884138,Walk Up,882,classic,2017-09-07 19:11:26+00:00,Capital Metro HQ - East 5th at Broadway,Capital Metro HQ - East 5th at Broadway,292,4.87


###**Pandas**

In [2]:
import pandas as pd

#####Data Loading

In [27]:
# working area
df = client.query('''
  SELECT *
  FROM `ftds-hacktiv8-project.practice.bike_rent`;
''').to_dataframe()

df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60
...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55


#####1. Cek kesesuaian tipe data dan keberadaan missing values dengan `.info()`

In [30]:
df.info()
# df['bike_id'].astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             8000 non-null   object             
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(2), datetime64[us, UTC](1), object(5)
memory usage: 515.8+ KB


Ketika di cek menggunakan df.info(), tipe data yang tidak cocok adalah `bike_id`. Ketika column `bike_id` di cek tidak ada missing value. Namun saat akan di convert menjadi `int` muncul error. Sehingga kemungkinan besar terdapat missing value yang tidak sesuai standard.

In [38]:
# working area
df_test = df

df_test['bike_id'] = pd.to_numeric(df_test['bike_id'], errors='coerce').astype('Int64')

df_test[df_test[['bike_id']].isna().any(axis=1)]

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   trip_id             8000 non-null   Int64              
 1   subscriber_type     8000 non-null   object             
 2   bike_id             7985 non-null   Int64              
 3   bike_type           8000 non-null   object             
 4   start_time          8000 non-null   datetime64[us, UTC]
 5   start_station_name  8000 non-null   object             
 6   end_station_name    8000 non-null   object             
 7   duration_minutes    8000 non-null   Int64              
dtypes: Int64(3), datetime64[us, UTC](1), object(4)
memory usage: 523.6+ KB


Setelah di cek ternyata ada 15 missing value

#####2. Buat kolom baru bernama `quarter`.
  
  Lakukan ekstraksi waktu (quarter dalam tahun) berdasarkan kolom `start_time` dengan ketentuan berikut:

<div align="center">

  |Keterangan Waktu|Bulan|
  |--|--|
  |Q1|Januari, Februari, Maret|
  |Q2|April, Mei, Juni|
  |Q3|Juli, Agustus, September|
  |Q4|Oktober, November, Desember|

</div>

>**Notes:**
>- Dalam menjawab soal Pandas no. 2, **tidak boleh** menggunakan ekstraksi waktu `Series.dt.quarter`.
- Lakukan `looping` dan `conditional` dengan memanfaatkan ekstraksi bulan `Series.dt.month` terhadap kolom `start_time`, pastikan tipe datanya ialah `datetime` agar dapat dilakukan ekstraksi.
- Simpan hasil looping ke dalam suatu variabel, dan buat kolom baru berisikan variabel sebelumnya.

In [39]:
# working area
quarter = []
for date in df['start_time']:
  if date.month in (1, 2, 3):
    quarter.append('Q1')
  elif date.month in (4, 5, 6):
    quarter.append('Q2')
  elif date.month in (7, 8, 9):
    quarter.append('Q3')
  elif date.month in (10, 11, 12):
    quarter.append('Q4')

df['quarter'] = quarter
df

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,quarter
0,18896589,Explorer,93,classic,2018-10-28 14:16:14+00:00,Dean Keeton & Speedway,11th & Salina,178,Q4
1,13948351,Local365,315,classic,2017-03-07 12:37:33+00:00,Main Office,Main Office,198,Q1
2,16243803,Local365,877,classic,2017-10-21 09:49:48+00:00,Republic Square @ 5th & Guadalupe,Main Office,60,Q4
3,16222446,Walk Up,273,classic,2017-10-18 15:01:52+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q4
4,16222451,Walk Up,201,classic,2017-10-18 15:02:26+00:00,Bullock Museum @ Congress & MLK,Main Office,60,Q4
...,...,...,...,...,...,...,...,...,...
7995,15469480,Walk Up,327,classic,2017-07-22 20:17:46+00:00,Bullock Museum @ Congress & MLK,Bullock Museum @ Congress & MLK,55,Q3
7996,16130650,Walk Up,528,classic,2017-10-07 11:25:36+00:00,Congress & Cesar Chavez,Bullock Museum @ Congress & MLK,55,Q4
7997,14204138,Walk Up,235,classic,2017-03-27 13:19:02+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q1
7998,15037809,Walk Up,206,classic,2017-06-05 11:42:40+00:00,State Capitol @ 14th & Colorado,State Capitol @ 14th & Colorado,55,Q2


#####3. Tampilkan data perjalanan dari tipe member `Local31` dengan durasi perjalanan diatas 1 jam.
*(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)*

In [40]:
# working area
df[(df['subscriber_type'] == 'Local31') & (df['duration_minutes'] > 60)]

,trip_id,subscriber_type,bike_id,bike_type,start_time,start_station_name,end_station_name,duration_minutes,quarter


#####4. Berapa rata-rata durasi perjalanan dari tipe pelanggan `Walk Up` dengan tipe sepeda `classic` pada bulan `Maret` dari data yang diambil?
*(Jika tidak ada entry yang memenuhi kondisi ini tidak apa, tuliskan kode pandasnya dengan lengkap saja)*

In [41]:
# working area
df_filtered = df[(df['subscriber_type'] == 'Walk Up') & (df['bike_type'] == 'classic') & (df['start_time'].dt.month ==3)]
df_filtered['duration_minutes'].mean()

38.476584022038566

In [ ]:
df_1.to_csv('a_out.csv')